## Homework 08:  Parallel Programming 01

## Due Date: Apr 21, 2022, 04:00pm

#### Firstname Lastname: Tianlin Shan

#### E-mail: ts4271@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [ ]:
%%writefile sol08pr01.py

from mpi4py import MPI
import numpy as np
comm = MPI.COMM_WORLD     #communicator object which is used to communicate between processes
rank = comm.Get_rank()
size = comm.Get_size()
data = 0
if rank == 1:
    data = 1
    comm.send(data, dest=rank + 1, tag=11)
else:
    data = comm.recv(source=(rank + size - 1) % size, tag=11)
    data += rank * rank;
    comm.send(data, dest=(rank + 1) % size, tag=11 )
    if rank == 0:
        print('rev {} received at rank {}:'.format(data, rank))

Writing sol08pr01.py


In [ ]:
!mpirun  --allow-run-as-root -n 10 python3 sol08pr01.py

rev 285 received at rank 0:


In [ ]:
!mpirun --allow-run-as-root -n 15 python sol08pr01.py

rev 1015 received at rank 0:


In [ ]:
!mpirun --allow-run-as-root -n 20 python sol08pr01.py

rev 2470 received at rank 0:


In [ ]:
!mpirun --allow-run-as-root -n 25 python sol08pr01.py

rev 4900 received at rank 0:


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [ ]:
%%writefile sol08pr02.py

from io import BufferedRandom
from mpi4py import MPI
import sys
import numpy as np
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
buffer = [0.0,0.0,1.0]


if rank == 0:
    for line in sys.stdin:
        x = float(line)
        buffer[0] = x
        if x != 0.0:
            break

    for line in sys.stdin:
        y = float(line)
        buffer[1] = x
        if y != 0.0:
            break
    for i in range(5):
        buffer[2] = buffer[2] * buffer[0]
        comm.send(buffer, dest=1, tag=11)
        buffer = comm.recv(source=1, tag=11)
    print("p final result is {}".format(buffer[2]))

else:
    for i in range(5):
        buffer = comm.recv(source=0, tag=11)
        buffer[1] = buffer[2] / buffer[1]
        comm.send(buffer, dest= 0, tag=11)


Overwriting sol08pr02.py


In [ ]:
!mpirun --allow-run-as-root -n 2 python sol08pr02.py 